<a href="https://colab.research.google.com/github/Jarde01/ml-tools-and-examples/blob/master/16_class_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Named Entity Recognition with recent and improved version of the Annotated NER Dataset with better features

**Note: It is just a quick demonstration on 'How to use this dataset on Scikit for Named Entity Classification'**
**This is not for analytical concepts and other machine learning illustrations!**

Reading CSV in pandas has many ways. I suggest to use CSV as database to prevent large file handling errors.
One may use the follwing way to read CSV file using Python Pandas. Good Luck

In [1]:
!pip install -U keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [0]:
!pip install --upgrade wandb

In [0]:
!wandb login 2295caae2022fe6a431a755f74d1e9e8605848eb
import wandb
from wandb.keras import WandbCallback
wandb.init(project="16-class-ner")


In [0]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, f1_score

In [0]:
import keras
from keras.preprocessing.text import *
from keras.preprocessing import *
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer

In [134]:
df = pd.read_csv("ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)
df.dropna(inplace=True)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [135]:
print(len(df['sentence_idx'].unique()))
num_samples = len(df)
num_samples

35178


1050795

In [136]:
filtered_df = filtered_df.filter(['word', 'tag', 'sentence_idx'])
filtered_df.tail()

,word,tag,sentence_idx
253904,the,O,11615
253905,1990s,B-tim,11615
253906,.,O,11615
253907,Ethiopia,B-geo,11616
253908,'s,O,11616


In [0]:
def get_multilabel():
  sentences = []
  tags = []

  sentids = filtered_df['sentence_idx'].unique()

  curr_sent_id = filtered_df['sentence_idx'].iloc[0]
  tokens = []
  tag_tokens = []
  row = None
  for index, row in filtered_df.iterrows():

    sent_id = row['sentence_idx']

    if curr_sent_id == sent_id:
      tokens.append(row['word'])
      tag_tokens.append(row['tag'])
    else:
      curr_sent_id = sent_id
      sentences.append(" ".join(tokens))
      tags.append(tag_tokens)
      tokens = []
      tag_tokens = []

  # get the last sentence
  sentences.append(" ".join(tokens))
  tags.append(tag_tokens)# print(sentences)
  
  x, y, enc, tok = prepare_multilabel(sentences, tags)
  
  return x, y, enc, tok

def prepare_multilabel(sentences, tags):
  # prepare y data
  y = np.array(tags)
  enc = MultiLabelBinarizer()
  y = enc.fit_transform(y)
  print(y[0])
  print(y.shape)
  
  # prepare x data
  x = np.array(sentences)
  tok = Tokenizer()
  tok.fit_on_texts(x)
  x = tok.texts_to_sequences(x)
  x = sequence.pad_sequences(x, maxlen=100, padding='post')
  x = np.array(x)
  return x, y, enc, tok


In [0]:
def get_singlelabel():
  words = filtered_df['word'].values
  tags = filtered_df['tag'].values
  
  x, y, enc, tok = prepare_singlelabel(words, tags)
  return x, y, enc, tok
  
def prepare_singlelabel(sentences, tags):
  # prepare y
  y = np.array(tags)
  enc = MultiLabelBinarizer()
  y = enc.fit_transform(y)
  print(y[0])
  print(y.shape)
  
  # prepare x
  x = np.array(sentences)
  tok = Tokenizer()
  tok.fit_on_texts(x)
  x = tok.texts_to_sequences(x)
  x = sequence.pad_sequences(x, maxlen=1, padding='post')
  x = np.array(x)
  print(x[1])
  print(x.shape)
  return x, y, enc, tok

In [153]:
# x, y = get_multilabel()
x, y, enc, tok = get_singlelabel()

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
(253909, 15)
[3]
(253909, 1)


In [0]:
# print(tok.__dict__.keys())
num_words = len(tok.word_index)

In [164]:
# classes = filtered_df['tag'].unique()
num_classes = y.shape[1]
num_classes

15

Splitting Train and test data

In [165]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
print(x_train.shape)
print(y_train.shape)

(177736, 1)
(177736, 15)


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, GlobalMaxPooling1D
from keras.optimizers import SGD
from keras import backend as K
  

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

  
def get_model(num_classes, num_words, embed_out=64, dense_1_nodes=64, dropout_1=0.5):
    model = Sequential()
    model.add(Embedding(num_words+1, embed_out, input_length=1, name='embedding_layer'))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(dense_1_nodes, activation='relu', name='dense_1'))
    model.add(Dropout(dropout_1))
#     model.add(Dense(num_classes, activation='sigmoid')) # multilabel
    model.add(Dense(num_classes, activation='sigmoid', name='output_layer')) #single label
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc',f1_m,precision_m, recall_m])
    return model

In [0]:
run_name = "" # what are you testing with this run
notes = ""

#hyperparams
hyperparams = dict(
  embed_out =64,
  dense_1_nodes = 16,
  dropout_1=0.3,
)

config= {
    "epochs": 20,
    "num_classes":num_classes,
    "num_words": num_words,
    "batch_size":batch_size,
    **hyperparams
}

model = get_model(num_classes, num_words, **hyperparams)

wandb.init(project="16-class-ner", name=run_name, config=config, notes=notes)

histroy = model.fit(x_train, y_train, batch_size=config['batch_size'], epochs=config['epochs'], 
          validation_split=0.2, callbacks=[WandbCallback()])

scores = model.evaluate(x_test, y_test)
wandb.log({'test_'+metric: scores[index] for index, metric in enumerate(model.metrics_names)})
print("\n*** Finished! ***")

Train on 142188 samples, validate on 35548 samples
Epoch 1/20
142188/142188 [==============================] - 5s 36us/step - loss: 3.7755 - acc: 0.7705 - f1_m: 0.5082 - precision_m: 0.4295 - recall_m: 0.7251 - val_loss: 2.6596 - val_acc: 0.8628 - val_f1_m: 0.7307 - val_precision_m: 0.8563 - val_recall_m: 0.6374
Epoch 2/20
142188/142188 [==============================] - 3s 19us/step - loss: 2.2792 - acc: 0.9061 - f1_m: 0.7800 - precision_m: 0.8584 - recall_m: 0.7154 - val_loss: 2.0054 - val_acc: 0.9464 - val_f1_m: 0.8289 - val_precision_m: 0.8901 - val_recall_m: 0.7757
Epoch 3/20
142188/142188 [==============================] - 3s 19us/step - loss: 1.9339 - acc: 0.9447 - f1_m: 0.8398 - precision_m: 0.8925 - recall_m: 0.7931 - val_loss: 1.8743 - val_acc: 0.9429 - val_f1_m: 0.8546 - val_precision_m: 0.9039 - val_recall_m: 0.8105
Epoch 4/20
142188/142188 [==============================] - 3s 20us/step - loss: 1.8185 - acc: 0.9452 - f1_m: 0.8581 - precision_m: 0.9020 - recall_m: 0.8185 - 